In [1]:
from scipy import sparse
import csv
import sys
import os
import pandas
import numpy as np
import multiprocessing

In [2]:
X = []
Y = []
f1=open(unicode('./datasetPersonTab'))
f2=open(unicode('./datasetNNPTab'))
for i, line in enumerate(f1):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[4:])
    Y.append('Person')
size = len(X)
np.random.shuffle(X)
for i, line in enumerate(f2):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[4:])
    Y.append('No')
np.random.shuffle(X[size:])

In [ ]:
X_test = []
Y_test = []
f1=open(unicode('./datasetPersonTestTab'))
f2=open(unicode('./datasetNNPTestTab'))
for i, line in enumerate(f1):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[4:])
    Y.append('Person')
size = len(X)
np.random.shuffle(X)
for i, line in enumerate(f2):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[4:])
    Y.append('No')
np.random.shuffle(X[size:])

In [3]:
set_futures =[]
for i in range(27):
    set_futures.append(set())
for x in X+X_test:
    for i, x_i in enumerate(x):
        set_futures[i].add(x_i)

In [4]:
feature_dict = []
for i in range(27):
    feature_dict.append(dict())
    
for i in range(27):
    count = 0
    for x in set_futures[i]:
        feature_dict[i][x] = count
        count+=1

In [5]:
X_int=[]
for x in X:
    X_int.append(x) 
for i, x in enumerate(X):
    for j, x_j in enumerate(x):
        X_int[i][j]=feature_dict[j][x_j]

In [ ]:
X_int_test=[]
for x in X_test:
    X_int.append(x) 
for i, x in enumerate(X_test):
    for j, x_j in enumerate(x):
        X_int_test[i][j]=feature_dict[j][x_j]

In [6]:
from sklearn.externals import joblib

In [7]:
filename = './DiplomaDataPerson_X_int.joblib.pkl'
joblib.dump(X_int, filename, compress=9)

['./DiplomaDataPerson_X_int.joblib.pkl']

In [8]:
X_int_int = joblib.load(filename)

In [10]:
len(X_int_int)

608280

In [51]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [52]:
enc.fit_transform(X_int)

<608280x243312 sparse matrix of type '<type 'numpy.float64'>'
	with 16423560 stored elements in Compressed Sparse Row format>

In [53]:
X_sparse_matrix = enc.transform(X_int[:70000]+X_int[200000:330000])

In [56]:
X_sparse_matrix.shape

(200000, 243312)

In [57]:
import sklearn.ensemble  
RF = sklearn.ensemble.RandomForestClassifier(n_estimators=400, n_jobs=3)

In [ ]:
X_new = RF.fit_transform(X_sparse_matrix, Y[:70000]+Y[200000:330000])

DONE!!!

In [ ]:
Y_train =RF.predict(enc.transform(X_int[5000:5500]))

In [ ]:
f_n = 0
f_p = 0
t_n=0
t_p=0
for i, y in enumerate(Y_train):
    if y != Y[5000+i]:
        if y=='No':
            f_n+=1
            #print X[10000+i]
        else:
            f_p+=1
    elif y =='No':
        t_n+=1
    else:
        t_p+=1
recall = t_p*1./(t_p+f_n+1)
presision = t_p*1./(t_p+f_p+1)
print recall
print presision
print f_p, f_n, t_p, t_n
    

In [ ]:
F_measure = 2*presision*recall/(presision+recall)
print F_measure

In [ ]:
scores = cross_validation.cross_val_score(RF, X_sparse_matrix, Y[:300000], cv = 9,n_jobs=-1)

TO DO